# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева*

## Практикум 3*. Формат JSON и его обработка в рамках блока кода JavaScript: продолжение

Импортируем стандартный набор инструментов – модуль `requests` и функцию `BeautifulSoup`, а также модуль `re` для использования регулярных выражений, модуль `json` для обработки JSON-строк, библиотеку `pandas` для работы с датафреймом.

In [1]:
import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

Совсем недавно у нас было домашнее задание на парсинг страницы фильма «Не покидай...» с сайта www.kino-teatr.ru. Сайт некоммерческий, довольно дружелюбный, позволяет свободно выгружать информацию (но уже не всегда!).

У него есть одна особенность: число лайков и дизлайков, поставленных актерам пользователями, загружается на страницу динамически, то есть автоматически «подтягивается» с сервера при загрузке страницы в определенный момент времени. На практике это выливается в то, что найти нужную информацию по тэгам просто невозможно, ее нет в основном коде HTML. Как быть? Понять, как выглядит запрос данных, который отправляется на сервер, и выяснить, где хранятся нужные нам данные. Мы рассмотрим несложный случай, когда сайт забирает информацию из строки JSON, которая находится на странице, но внутри кода, написанного на JavaScript. Такое можно встретить на страницах с результатами каких-нибудь игр или на сайтах, посвященных динамике цен или курсу валют (другой вопрос, что не всегда JSON прямо так явно находится в том же файле, где и код HTML).

Так как ранее мы обсуждали довольно базовый парсинг, в домашнем задании не требовалось собирать число лайков и дизлайков, эти значения были даны в виде готового списка словарей. Обновим задачу – теперь нам нужно собрать только id актеров и информацию о числе голосов за и против.

Начало работы стандартное – загружаем код HTML страницы по ссылке и преобразуем его в объект BeautifulSoup:

In [2]:
# ссылка на копию страницы во избежание блокировок

page = requests.get("https://raw.githubusercontent.com/allatambov/WebScrape25/refs/heads/main/titr.html")
soup = BeautifulSoup(page.text)

Просто найти на странице кнопки красного и зеленого цвета и забрать с них текст не получится. Поэтому для этого на нужно найти код JavaScript, где есть записи с числами `plus` и `minus` с привязкой к id актеров. Код JavaScript, если он не вынесен в отдельный файл, заключается в тэги `<script>`. Если мы внимательно изучим исходный код страницы, мы заметим, что нужный нам блок кода имеет атрибут `type` со значением `text/javascript`, и среди таких блоков он находится на 6-ом месте:

In [3]:
# выполняем поиск и извлекаем текст – код JavaScript в виде строки

text = soup.find_all("script", {"type" : "text/javascript"})[5].next
#print(text)

In [4]:
for_votes = text.split("function print_all_role_rating_buttons ()")[1]
print(for_votes)


{
    print_role_rating_buttons({ id:"160325", plus:"64", minus:"3", voted:"" });
print_role_rating_buttons({ id:"1928641", plus:"33", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1966145", plus:"29", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973000", plus:"30", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973001", plus:"30", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973002", plus:"31", minus:"0", voted:"" });
print_role_rating_buttons({ id:"1973003", plus:"30", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973005", plus:"30", minus:"2", voted:"" });
print_role_rating_buttons({ id:"1973007", plus:"33", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973008", plus:"28", minus:"1", voted:"" });
print_role_rating_buttons({ id:"1973010", plus:"38", minus:"7", voted:"" });
print_role_rating_buttons({ id:"1973011", plus:"49", minus:"3", voted:"" });
print_role_rating_buttons({ id:"2030314", plus:"2", minus:"0", voted:"

Хотя с синтаксисом JavaScript знакомы немногие, не очень сложно догадаться, что означают записи такого вида:

    print_role_rating_buttons({ id:"16800", plus:"173", minus:"4", voted:"" });

Такая строка кода активирует функцию print_role_rating_buttons() – применяет ее к такому набору данных и наносит на кнопки, соответствующие актеру с id 16800 значения 173 (зеленая кнопка) и 4 (красная кнопка).

*Примечание 1.* Синхронизация не происходит мгновенно, актуальное значение 173 подставляется на кнопку не сразу, там вполне может стоять значение 172, а мы, благодаря значениям из JavaScript, обладаем более актуальной информацией. Это стоит иметь в виду при выгрузке информации, когда мы сверяем полученные результаты и то, что видим на экране.

*Примечание 2.* Значение `voted` в коде выше является непустым, в случае, если пользователь зарегистрирован и оценивал игру актера (по крайней мере, какое-то время после голосования значение в `voted` держится).

Объект `for_votes` – это обычная строка, объект типа `string`. Поэтому с помощью регулярных выражений мы сможем найти в ней записи в фигурных скобках, затем сделать из них словари, а из списка словарей с единообразными ключами собрать датафрейм!

### Задача 1

Найдите все подстроки, соответствующие наборам символов, заключенных в фигурные скобки – чтобы из них можно было собрать словари. Сохраните список строк в переменную `votes_str`.

In [5]:
votes_str = re.findall("\{.+\}", for_votes)
votes_str

['{ id:"160325", plus:"64", minus:"3", voted:"" }',
 '{ id:"1928641", plus:"33", minus:"1", voted:"" }',
 '{ id:"1966145", plus:"29", minus:"1", voted:"" }',
 '{ id:"1973000", plus:"30", minus:"1", voted:"" }',
 '{ id:"1973001", plus:"30", minus:"1", voted:"" }',
 '{ id:"1973002", plus:"31", minus:"0", voted:"" }',
 '{ id:"1973003", plus:"30", minus:"1", voted:"" }',
 '{ id:"1973005", plus:"30", minus:"2", voted:"" }',
 '{ id:"1973007", plus:"33", minus:"1", voted:"" }',
 '{ id:"1973008", plus:"28", minus:"1", voted:"" }',
 '{ id:"1973010", plus:"38", minus:"7", voted:"" }',
 '{ id:"1973011", plus:"49", minus:"3", voted:"" }',
 '{ id:"2030314", plus:"2", minus:"0", voted:"" }',
 '{ id:"2088754", plus:"35", minus:"2", voted:"" }',
 '{ id:"16800", plus:"173", minus:"4", voted:"" }',
 '{ id:"16801", plus:"119", minus:"28", voted:"" }',
 '{ id:"16803", plus:"136", minus:"12", voted:"" }',
 '{ id:"16802", plus:"158", minus:"11", voted:"" }',
 '{ id:"16804", plus:"150", minus:"6", voted:"" }

Выберем один элемент списка и изучим его:

In [6]:
v = votes_str[0]
print(v)

{ id:"160325", plus:"64", minus:"3", voted:"" }


На первый взгляд, этот элемент представляет собой полноценный словарь, просто заключенный в кавычки. Однако есть проблема: чтобы переделать строку в словарь, нужна строка, соответствующая формату JSON, а здесь не хватает кавычек вокруг ключей. Как эту проблему решить? Найти по какому-то паттерну эти ключи и доклеить вокруг них кавычки с помощью функции `sub()` из модуля `re` для «умной» замены (не обычная замена, так как мы не конкретный набор символов заменяем на другой, а ищем совпадения по некоторому общему шаблону и их изменяем).

Поиск будет простой – в строке с «неправильным» словарем содержатся либо цифры, либо буквы, либо пробелы с запятыми и знаками препинания, а нам нужны последовательности *из одной и более букв*. 

### Задача 2

Напишите регулярное выражение, которое в строке `v` найдет все слова из букв.

In [7]:
re.findall("[a-z]+", v)

['id', 'plus', 'minus', 'voted']

Напишем функцию `add_quotes()`, которая принимает на вход строку `x`, доклеивает к ней кавычки и убирает пробелы. Нам нужно преобразовать строку `x` в группу – объект из регулярных выражений – через `+` доклеить кавычки и на всякий случай убрать лишние пробелы:

In [8]:
# x.group(): функция sub() из re для умной замены
# работает не со строками, а с объектом специального типа – группа символов

def add_quotes(x):
    r = '"' + x.group() + '"'
    return r.strip()

In [9]:
re.sub("[a-z]+", add_quotes, v)

'{ "id":"160325", "plus":"64", "minus":"3", "voted":"" }'

Применяем функцию `sub` к одной строке `v` и проверяем:

In [10]:
# аргументы:
# шаблон, по которому ищем, что заменять
# функция, которая выполняет преобразование – доклеивает к результатм поиска кавычки
# строка, по который выполняем поиск

re.sub("[a-z]+", add_quotes, v)

'{ "id":"160325", "plus":"64", "minus":"3", "voted":"" }'

**NB.** Если парсить вариант страницы на Github, в некоторых строках в `voted` не пустая строка, а тоже слово – `plus`. Поэтому операция выше на этих элементах будет выдавать текст с лишними кавычками внутри, что нельзя будет интерпретировать как валидные JSON-строки. Пример:

In [11]:
re.sub("[a-z]+", add_quotes, votes_str[-4]) # "voted":""plus""

'{ "id":"132139", "plus":"124", "minus":"5", "voted":""plus"" }'

Перед дальнейшими манипуляциям с модулем `json` избавимся от этих повторяющихся кавычек с помощью обычной замены через метод `.replace()` на строках:

In [12]:
# заменяем ""plus"" на "plus"
# были бы минусы, проделали тоже и для ""minus""
# просто заменить дублирующися кавчки нельзя, в voted может быть
# пустая строка, и тогда нужные кавычки тоже слетят

new = [re.sub("[a-z]+", add_quotes, v).replace('""plus""', '"plus"') for v in votes_str]
new

['{ "id":"160325", "plus":"64", "minus":"3", "voted":"" }',
 '{ "id":"1928641", "plus":"33", "minus":"1", "voted":"" }',
 '{ "id":"1966145", "plus":"29", "minus":"1", "voted":"" }',
 '{ "id":"1973000", "plus":"30", "minus":"1", "voted":"" }',
 '{ "id":"1973001", "plus":"30", "minus":"1", "voted":"" }',
 '{ "id":"1973002", "plus":"31", "minus":"0", "voted":"" }',
 '{ "id":"1973003", "plus":"30", "minus":"1", "voted":"" }',
 '{ "id":"1973005", "plus":"30", "minus":"2", "voted":"" }',
 '{ "id":"1973007", "plus":"33", "minus":"1", "voted":"" }',
 '{ "id":"1973008", "plus":"28", "minus":"1", "voted":"" }',
 '{ "id":"1973010", "plus":"38", "minus":"7", "voted":"" }',
 '{ "id":"1973011", "plus":"49", "minus":"3", "voted":"" }',
 '{ "id":"2030314", "plus":"2", "minus":"0", "voted":"" }',
 '{ "id":"2088754", "plus":"35", "minus":"2", "voted":"" }',
 '{ "id":"16800", "plus":"173", "minus":"4", "voted":"" }',
 '{ "id":"16801", "plus":"119", "minus":"28", "voted":"" }',
 '{ "id":"16803", "plus":"1

### Задача 3

Примените операцию выше ко всем строкам в `votes_str` и получите список валидных JSON-строк. Используя модуль `json`, превратите список JSON-строк в список питоновских словарей. Создайте на его основе датафрейм pandas.

In [13]:
# превращаем каждую строку в new в полноценный словарь,
# десериализуя json-строки через loads()

L_dicts = [json.loads(s) for s in new]
L_dicts

[{'id': '160325', 'plus': '64', 'minus': '3', 'voted': ''},
 {'id': '1928641', 'plus': '33', 'minus': '1', 'voted': ''},
 {'id': '1966145', 'plus': '29', 'minus': '1', 'voted': ''},
 {'id': '1973000', 'plus': '30', 'minus': '1', 'voted': ''},
 {'id': '1973001', 'plus': '30', 'minus': '1', 'voted': ''},
 {'id': '1973002', 'plus': '31', 'minus': '0', 'voted': ''},
 {'id': '1973003', 'plus': '30', 'minus': '1', 'voted': ''},
 {'id': '1973005', 'plus': '30', 'minus': '2', 'voted': ''},
 {'id': '1973007', 'plus': '33', 'minus': '1', 'voted': ''},
 {'id': '1973008', 'plus': '28', 'minus': '1', 'voted': ''},
 {'id': '1973010', 'plus': '38', 'minus': '7', 'voted': ''},
 {'id': '1973011', 'plus': '49', 'minus': '3', 'voted': ''},
 {'id': '2030314', 'plus': '2', 'minus': '0', 'voted': ''},
 {'id': '2088754', 'plus': '35', 'minus': '2', 'voted': ''},
 {'id': '16800', 'plus': '173', 'minus': '4', 'voted': ''},
 {'id': '16801', 'plus': '119', 'minus': '28', 'voted': ''},
 {'id': '16803', 'plus': '1

In [14]:
# превращаем список словарей в датафрейм

df = pd.DataFrame(L_dicts)
df

,id,plus,minus,voted
0,160325,64,3,
1,1928641,33,1,
2,1966145,29,1,
3,1973000,30,1,
4,1973001,30,1,
5,1973002,31,0,
6,1973003,30,1,
7,1973005,30,2,
8,1973007,33,1,
9,1973008,28,1,


In [15]:
# меняем тип столбцов plus и minus на integer

df["plus"] = df["plus"].astype(int)
df["minus"] = df["minus"].astype(int)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      30 non-null     object
 1   plus    30 non-null     int64 
 2   minus   30 non-null     int64 
 3   voted   30 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.1+ KB
